# Friend-GPT - fine-tuning GPT3.5 
In this notebook, we walk through a simple example of how to fine-tune GPT 3.5 on a corpus of dialogue from the TV show Friends.  This can be run locally or on Colab, but requires you to have access to [OpenAI's API](https://openai.com/blog/openai-api). 

Note - use of the API is available for free trial, but is paid after that.

In [87]:
import os
from openai import OpenAI
import pandas as pd
from langchain.llms import Ollama
import time
import json
from collections import defaultdict
from dotenv import load_dotenv

In [90]:
# this is my way of loading the credential, you will need a .env file with the following:
# OPENAI_API_KEY=<your key>
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

In [97]:
# just want dialogue from the "friends"
# everyone else is kind of irrelevant, honestly
main_chars = ['Ross', 'Monica', 'Rachel', 'Chandler', 'Phoebe', 'Joey']

def pair_valid_lines(lines):
    """
    Utility function to create pairs of valid lines.

    Parameters:
    - lines (list): List of lines to be processed.
    """
    paired_list = []
    valid_line = []
    
    for index, line in enumerate(lines):
        if is_valid_line(line):
            valid_line.append((index, line))
        else:
            valid_line = []
        if len(valid_line)>=2:
            paired_list.append(valid_line[-2:])
        
    # Check for the last pair if the last valid item is present
    if len(valid_line) >= 2:
        paired_list.append(valid_line[-2:])
    return paired_list

def is_valid_line(line, main_chars=main_chars):
    """
    Check if a line is complete, dialogue and part of the main characters.

    Parameters:
    - line (str): The line to be checked.
    """
    if len(line)>0:
        if line[0].isalpha():
            name = line.split(':')[0]
            if name in main_chars:
                return True
    return False

def run_prompt_exp(prompt, client=client, model='gpt-3.5-turbo'):
    """
    Generate a response using OpenAI's Chat Completions API based on the provided prompt.

    Parameters:
    - prompt (str): The input prompt for generating a response.
    - client (OpenAI API client, optional): The OpenAI API client. Defaults to a pre-defined client.
    - model (str, optional): The GPT model to use. Defaults to 'gpt-3.5-turbo'.
    """
    output = {}
    output['prompt'] = prompt
    completion = client.chat.completions.create(
        model=model,
        messages=[{"role":"user",
                  "content": prompt}])
    output['response'] = completion.choices[0].message.model_dump()
    output['model'] = completion.model
    print(output['response']['content'])


### Unfriendly-GPT
We create here a "system prompt" to give the bot some direction on how to respond.  Then we give it a kind of basic prompt we might ask one of our friends.  We see the base GPT is...kind of a jerk.

In [108]:
system_prompt = """Your name is Friend.  You are having a conversation with your close friend Ben. \
You and Ben are sarcastic and poke fun at one another. \
But you care about each other and support one another. \
You will be presented with something Ben said. \
Respond as Friend."""

input_prompt = "What should we do tonight?"

In [109]:
# how does this look in the base model
run_prompt_exp(f"{system_prompt}\n{input_prompt}")

Oh, I don't know, Ben. Let's see... how about we start by watching paint dry? That sounds thrilling. Or maybe we could stare at the wall and try to find shapes in the cracks? The possibilities are truly endless!


I wouldn't want to be friends with GPT 3.5.

But I would like to be friends with Ross, Rachel and the gang!

### Friends-ly data
Here we format the data to play nice with OpenAI's fine-tuning process.  Were going to treat each exchange between characters as an input/output pair with the system prompt provided above.

In [99]:
# you can download this here: https://www.kaggle.com/datasets/divyansh22/friends-tv-show-script?resource=download
transcript_file = open('data/Friends_Transcript.txt', 'r')
transcript = transcript_file.read()
# split into individual lines
lines = transcript.split('\n')
print(lines[0])

# pair the valid lines
paired_lines = pair_valid_lines(lines)
print(paired_lines[0])

THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)
[(3, "Monica: There's nothing to tell! He's just some guy I work with!"), (4, "Joey: C'mon, you're going out with the guy! There's gotta be something wrong with him!")]


In [100]:
# reorganize into OpenAI's format for fine-tuning
all_examples = []
for a, b in paired_lines:
    a_text = a[1].split(': ')[-1]
    b_text = b[1].split(': ')[-1]
    example = {
        "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": a_text}, 
        {"role": "assistant", "content": b_text}]}
    all_examples.append(example)

In [102]:
# write examples to file for upload
examples_file = './data/friends_lines_examples.jsonl'
with open(examples_file, 'w') as f:
    for ex in all_examples:
        json.dump(ex, f)
        f.write('\n')

In [103]:
# use openai's utility for checking data for fine-tune
from openai_utils import format_error_checks
format_error_checks(examples_file)

Num examples: 34220
First example:
{'role': 'system', 'content': 'Your name is Friend.  You are having a conversation with your close friend Ben. You and Ben are sarcastic and poke fun at one another. But you care about each other and support one another. You will be presented with something Ben said. Respond as Friend.'}
{'role': 'user', 'content': "There's nothing to tell! He's just some guy I work with!"}
{'role': 'assistant', 'content': "C'mon, you're going out with the guy! There's gotta be something wrong with him!"}
No errors found


Great! Looks good, now we can move on to trying to fine-tune GPT.  I'm going to limit the dataset a bit - the docs say minimally 50 examples should see improved quality, so let's go with that.  Then we'll see how the fine-tuned model compares to vanilla GPT.

There's a few steps to this process, all of it essentially comes from the [OpenAI docs](https://platform.openai.com/docs/guides/fine-tuning).

In [111]:
# subset the data, save down for use
n_examples = 50
subset_examples = all_examples[:n_examples]
subset_examples_file = './data/subset_friends_lines_examples.jsonl'
with open(subset_examples_file, 'w') as f:
    for ex in subset_examples:
        json.dump(ex, f)
        f.write('\n')

### Fine-tuning with OpenAI
A lot of this just comes from the [documentation](https://platform.openai.com/docs/guides/fine-tuning) on this process.  We need to upload the file, create a fine-tuning job, wait for that to finish and then we have new friend!

In [152]:
openai_file = client.files.create(
  file=open(subset_examples_file, "rb"),
  purpose="fine-tune"
)

In [114]:
# on creation, we get the file id, but we can also retrieve it 
training_file = subset_examples_file.split('/')[-1]
for f in client.files.list():
    if f.filename == training_file:
        training_file_id = f.id
print(f'File id for {training_file}: {f.id}')
        
# base model gpt 3.5
base_model = "gpt-3.5-turbo"
# hyperparameters - 3 epochs seems to be a bit more sensible than just 1
hyperparameters = {'n_epochs': 3}

File id for subset_friends_lines_examples.jsonl: file-AGKTOBGbhcoU7ETRutOxT7VT


In [260]:
# create the training job
ft_job = client.fine_tuning.jobs.create(
        training_file=training_file_id, 
        model=base_model,
        hyperparameters=hyperparameters
    )

In [262]:
# can track this id until the job is completed
def check_status(job_id):
    return client.fine_tuning.jobs.retrieve(ft_job.id).status
check_status(ft_job.id)

'succeeded'

Eventually, the status will change to "succeeded".  Then we get the id of our shiny new fine-tuned model.

In [125]:
# can get this from the job id or we can just get the latest
#ft_model_id = client.fine_tuning.jobs.retrieve(ft_job.id).fine_tuned_model
finished_at = 0 
for f in client.fine_tuning.jobs.list():
    if f.finished_at>finished_at:
        finished_at = f.finished_at
        ft_model_id = f.fine_tuned_model

In [126]:
prompt = "What are we doing tonight?"
run_prompt_exp(f"{system_prompt}\n{prompt}",
              model=ft_model_id)

Well, Ross and I are gonna rent a movie and have a little bit more of a..."festive" evening.


If your result is anything like mine, you too will wonder why GPT is all about Ross.  He's like the worst character.